<a href="https://colab.research.google.com/github/SunwungLee/tensorflow-study/blob/main/tf_tutorial_7_save_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
모델 저장과 복원

작성자: 이선웅
작성일: 2021.07.06
'''

'\n모델 저장과 복원\n\n작성자: 이선웅\n작성일: 2021.07.06\n'

# 모델 저장과 복원

훈련하는 도중이나 훈련이 끝난 후에 모델을 저장할 수 있습니다. 모델을 중지된 지점부터 다시 훈련할 수 있어 한 번에 오랫동안 훈련하지 않아도 됩니다. 또 모델을 저장하면 다른 사람에게 공유할 수 있고 작업을 재현할 수 있습니다. 연구한 모델과 기법을 공개할 때 많은 머신 러닝 기술자들이 다음과 같은 것들을 제공합니다:

- 모델을 만드는 코드
- 모델의 훈련된 가중치 또는 파라미터

이런 데이터를 공유하면 다른 사람들이 모델의 작동 방식을 이해하고 새로운 데이터로 모델을 실험하는데 도움이 됩니다.

주의: 신뢰할 수 없는 코드는 조심하세요. 텐서플로 모델은 프로그램 코드입니다. 자세한 내용은 텐서플로를 안전하게 사용하기 문서를 참고하세요.

###저장 방식
사용하는 API에 따라서 여러가지 방법으로 텐서플로 모델을 저장할 수 있습니다. 이 문서는 텐서플로 모델을 만들고 훈련하기 위한 고수준 API인 tf.keras를 사용합니다. 다른 방법들에 대해서는 텐서플로의 저장과 복원 문서와 즉시 실행(eager execution) 문서의 저장하기 섹션을 참고하세요.

##설정
###설치와 임포트
필요한 라이브러리를 설치하고 텐서플로를 임포트(import)합니다:

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.5.0


### 예제 데이터셋 받기
MNIST 데이터셋으로 모델을 훈련하여 가중치를 저장하는 예제를 만들어 보겠습니다. 모델 실행 속도를 빠르게 하기 위해 샘플에서 처음 1,000개만 사용겠습니다:

In [ ]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


### 모델 정의
먼저 간단한 모델을 하나 만들어 보죠.

In [ ]:
def create_model():
    model = keras.models.Sequential(
        [
         keras.layers.Dense(512, activation='relu', input_shape=(784, )),
         keras.layers.Dropout(0.2),
         keras.layers.Dense(10)
        ]
    )
    model.compile(
        optimizer='adam',
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    return model

model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## 훈련하는 동안 체크포인트 저장하기
훈련 중간과 훈련 마지막에 체크포인트(checkpoint)를 자동으로 저장하도록 하는 것이 많이 사용하는 방법입니다. 다시 훈련하지 않고 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행할 수 있습니다. tf.keras.callbacks.ModelCheckpoint은 이런 작업을 수행하는 콜백(callback)입니다. 이 콜백은 체크포인트 작업을 조정할 수 있도록 여러가지 매개변수를 제공합니다.

### 체크포인트 콜백 사용하기
훈련하는 동안 가중치를 저장하기 위해 ModelCheckpoint 콜백을 만들어 보죠:

In [ ]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1
)

model.fit(
    train_images,
    train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    callbacks=[cp_callback]
)

Epoch 1/10
32/32 [==============================] - 1s 16ms/step - loss: 1.1395 - accuracy: 0.6780 - val_loss: 0.7368 - val_accuracy: 0.7710

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 11ms/step - loss: 0.4221 - accuracy: 0.8810 - val_loss: 0.5767 - val_accuracy: 0.8120

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 9ms/step - loss: 0.2773 - accuracy: 0.9220 - val_loss: 0.4954 - val_accuracy: 0.8410

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 11ms/step - loss: 0.2099 - accuracy: 0.9480 - val_loss: 0.4467 - val_accuracy: 0.8590

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 8ms/step - loss: 0.1622 - accuracy: 0.9620 - val_loss: 0.4295 - val_accuracy: 0.8710

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
32/32 [==============================] - 

In [ ]:
ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


훈련하지 않은 새로운 모델을 만들어 보겠습니다. 가중치만 복원할 땐 원본 모델과 동일한 구조로 모델을 만들어야 합니다. 여기서는 동일한 구조로 모델을 만들었으므로 다른 객체이지만 가중치를 공유할 수 있습니다.

훈련하지 않은 새 모델을 만들고 테스트 세트에서 평가해 보죠. 훈련되지 않은 모델의 성능은 무작위로 선택하는 정도의 수준입니다(~10% 정확도):

In [ ]:
model = create_model()

loss, acc = model.evaluate(
    test_images,
    test_labels,
    verbose=2
)
print(f'훈련되지 않은 모델의 정확도: {100*acc}%')

32/32 - 0s - loss: 2.3461 - accuracy: 0.0950
훈련되지 않은 모델의 정확도: 9.49999988079071%


In [ ]:
model.load_weights(checkpoint_path)

loss,acc = model.evaluate(
    test_images,
    test_labels,
    verbose=2
)
print(f'복원된 모델의 정확도: {100*acc}%')

32/32 - 0s - loss: 0.3949 - accuracy: 0.8750
복원된 모델의 정확도: 87.5%


### 체크포인트 콜백 매개변수
이 콜백 함수는 몇 가지 매개변수를 제공합니다. 체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정할 수 있습니다.

새로운 모델을 훈련하고 다섯 번의 에포크마다 고유한 이름으로 체크포인트를 저장해 보겠습니다:

In [ ]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(
    train_images,
    train_labels,
    epochs=50,
    callbacks=[cp_callback],
    validation_data=(test_images, test_labels),
    verbose=0
)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [ ]:
ls {checkpoint_dir}

checkpoint                        cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index                cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index                cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index                cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index                cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index                cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

노트: 텐서플로는 기본적으로 최근 5개의 체크포인트만 저장합니다.

모델을 초기화하고 최근 체크포인트를 로드하여 테스트해 보겠습니다:

In [ ]:
model = create_model()
model.load_weights(latest)

loss, acc = model.evaluate(
    test_images,
    test_labels,
    verbose=2
)
print(f'복원된 모델의 정확도: {100*acc}%')

32/32 - 0s - loss: 0.4875 - accuracy: 0.8750
복원된 모델의 정확도: 87.5%


## 이 파일들은 무엇인가요?
The above code stores the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain:

- One or more shards that contain your model's weights.
- An index file that indicates which weights are stored in a which shard.
If you are only training a model on a single machine, you'll have one shard with the suffix: .data-00000-of-00001


## 수동으로 가중치 저장하기
앞에서 가중치를 모델에 로드하는 방법을 보았습니다. 수동으로 가중치를 저장하는 것도 쉽습니다. Model.save_weights 메서드를 사용합니다. tf.keras는, 특히 save_weights는 .ckpt 확장자를 가진 텐서플로 체크포인트 포맷을 사용합니다(.h5 확장자의 HDF5으로 저장하는 것은 Save and serialize models 가이드에서 다룹니다):

In [ ]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(
    test_images,
    test_labels,
    verbose=2
)
print(f'복원된 모델의 정확도: {100*acc}%')

32/32 - 0s - loss: 0.4875 - accuracy: 0.8750
복원된 모델의 정확도: 87.5%


## 전체 모델 저장하기
model.save 메서드를 호출하여 모델의 구조, 가중치, 훈련 설정을 하나의 파일/폴더에 저장합니다. 모델을 저장하기 때문에 원본 파이썬 코드*가 없어도 사용할 수 있습니다. 옵티마이저 상태가 복원되므로 정확히 중지한 시점에서 다시 훈련을 시작할 수 있습니다.

두 개의 포맷(SavedModel과 HDF5)으로 모델을 저장할 수 있습니다. 텐서플로의 SavedModel 포맷은 TF2.x에서기본 파일 포맷입니다. 하지만 HDF5 포맷으로 저장할 수도 있습니다. 두 파일 포맷으로 전체 모델을 저장하는 방법은 아래에서 자세히 설명합니다.

전체 모델을 저장하는 기능은 매우 유용합니다. TensorFlow.js로 모델을 로드한 다음 웹 브라우저에서 모델을 훈련하고 실행할 수 있습니다(Saved Model, HDF5). 또는 모바일 장치에 맞도록 변환한 다음 TensorFlow Lite를 사용하여 실행할 수 있습니다(Saved Model, HDF5).

* 사용자 정의 객체(예를 들면 상속으로 만든 클래스나 층)는 저장하고 로드하는데 특별한 주의가 필요합니다. 아래 사용자 정의 객체 저장하기 섹션을 참고하세요.

### SavedModel 포맷
SavedModel 포맷은 모데을 직렬화하는 다른 방법입니다. 이 포맷으로 저장한 모델은 tf.keras.models.load_model로 복원할 수 있고 텐서플로 서빙과 호환됩니다. SavedModel 가이드에서 SavedModel를 서빙하고 점검하는 자세한 방법을 제공합니다. 이 섹션에서는 모델을 저장하고 복원하는 방법을 안내하겠습니다.

In [ ]:
model = create_model()
model.fit(
    train_images,
    train_labels,
    epochs=5
)

!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.1870 - accuracy: 0.6590
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4286 - accuracy: 0.8870
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2892 - accuracy: 0.9210
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2080 - accuracy: 0.9500
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1512 - accuracy: 0.9680
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
ls saved_model/my_model/

assets/  keras_metadata.pb  saved_model.pb  variables/


In [ ]:
new_model = keras.models.load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4229 - accuracy: 0.8610
복원된 모델의 정확도: 86.10%
(1000, 10)


### HDF5 파일로 저장하기
케라스는 HDF5(Hierarchical Data Format) 표준을 따르는 기본 저장 포맷을 제공합니다.

In [ ]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.1696 - accuracy: 0.6730
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4250 - accuracy: 0.8780
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2951 - accuracy: 0.9210
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2298 - accuracy: 0.9410
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1529 - accuracy: 0.9700


In [ ]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model('my_model.h5')

# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4273 - accuracy: 0.8660
복원된 모델의 정확도: 86.60%


케라스는 모델 구조를 파악하여 저장합니다. 이 방식은 다음과 같은 모든 것을 저장합니다:

- 가중치 값
- 모델 구조
- 모델의 훈련 설정(컴파일할 때 전달한 값)
- 옵티마이저와 필요하면 옵티마이저 상태(중지된 지점부터 다시 시작할 수 있습니다)

체크포인트가 호환되지 않기 때문에 케라스는 v1.x 옵티마이저(tf.compat.v1.train)를 저장할 수 없습니다. v1.x 옵티마이저를 사용하려면 로드한 후에 모델을 다시 컴파일해야 합니다. 따라서 옵티마이저의 상태를 잃게 됩니다.

### 사용자 정의 객체
SaveModel 포맷을 사용한다면 이 섹션은 건너 뛰어도 됩니다. HDF5와 SavedModel 사이의 주요 차이점은 HDF5가 객체로 모델 구조를 저장하고 SavedModel은 실행 그래프로 저장하는 것입니다. 따라서 SavedModel은 원본 코드 없이 상속 모델과 사용자 정의 층 같은 사용자 정의 객체를 저장할 수 있습니다.

사용자 정의 객체를 HDF5로 저장하려면 다음 과정을 따르세요:

1. 이 객체에 get_config 메서드를 정의하고 선택적으로 from_config 클래스 메서드를 정의합니다.
    - get_config(self)는 객체를 다시 생성하기 위해 필요한 JSON 직렬화된 매개변수 딕셔너리를 반환합니다.
    - from_config(cls, config)는 get_config에서 반환된 설정을 사용해 새로운 객체를 만듭니다. 기본적으로 이 함수는 이 설정을 초기화 메서드의 매개변수로 사용합니다(return cls(**config)).
2. 모델을 로드할 때 이 객체를 custom_objects 매개변수로 전달합니다. 문자열 클래스 이름과 파이썬 클래스를 매핑한 딕서너리를 매개변수로 제공해야 합니다. 예를 들면 tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})

사용자 정의 객체와 get_config에 관한 예제를 보려면 Writing layers and models from scratch 튜토리얼을 참고하세요.

In [ ]:
# MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.